In [1]:
import requests
from dotenv import load_dotenv
import os
      
load_dotenv()

onemap_api_key = os.environ.get("ONEMAP_API_KEY")


onemap_headers = {
    'User-Agent': onemap_api_key
}


In [2]:
import pandas as pd

In [3]:


def onemap_search(searchquery):
    url = "https://www.onemap.gov.sg/api/common/elastic/search"
    params = {
        "searchVal": searchquery,
        "returnGeom": "Y",
        "getAddrDetails": "Y"
    }

    response = requests.get(url, params=params, headers=onemap_headers)
    
    if response.status_code == 200:
        data = response.json()
        if data.get('found') > 0:
            result = data['results'][0]  # Assuming you want the first result
            latitude = result['LATITUDE']
            longitude = result['LONGITUDE']
            return latitude, longitude
        else:
            print(searchquery + "    No results found.")
            return None, None
    else:
        print(searchquery + "    Request failed with status code:", response.status_code)
        return None, None


In [12]:
all_gp = pd.read_csv("..\\1.5 FixingDataIssues\\All GPs.csv", index_col=0)
all_hospital = pd.read_csv("..\\1.5 FixingDataIssues\\All Hospitals.csv", index_col=0)
all_poly = pd.read_csv("..\\1.5 FixingDataIssues\\All Polyclinics.csv", index_col=0)


In [13]:
# function to perform on each row
def helper(row):
    return onemap_search(row['Postal Code'])


In [16]:
all_gp['Postal Code'] = all_gp['Location'].str.extract(r'Singapore (\d+)')
all_gp[["Lat", "Long"]] = all_gp.apply(lambda row: pd.Series(helper(row)), axis = 1, result_type='expand')
all_gp.to_csv(".\\Output Institution\\All GPs.csv")

279887    No results found.
828273    No results found.
760102    No results found.


In [14]:
all_hospital['Postal Code'] = all_hospital['Location'].str.extract(r'Singapore (\d+)')
all_hospital[["Lat", "Long"]] = all_hospital.apply(lambda row: pd.Series(helper(row)), axis = 1, result_type='expand')
all_hospital.to_csv(".\\Output Institution\\All Hospitals.csv")

In [15]:
all_poly['Postal Code'] = all_poly['Location'].str.extract(r'Singapore (\d+)')
all_poly[["Lat", "Long"]] = all_poly.apply(lambda row: pd.Series(helper(row)), axis = 1, result_type='expand')
all_poly.to_csv(".\\Output Institution\\All Polyclinics.csv")